## Collecting Data

To collect my Spotify Data I used this source: https://github.com/plamere/spotipy/tree/master/examples

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 


client_id = '3d5be77a0a8b4b34a19fe1c509719169'
client_secret = 'b5f6d556e9a74f48a88ec4e4a40bfc5e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)

    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
        
    
    return ids



ids = getTrackIDs('Mine1', '43Fiv11oJLTBKtc2VcpmIy')


def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  length = meta['duration_ms']
  popularity = meta['popularity']
 
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  mode = features[0]['mode']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, length, popularity, danceability, acousticness, energy, instrumentalness, mode, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track



# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
tracks_df = pd.DataFrame(tracks, columns = ['trackname', 'album', 'artist', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', ' mode', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [3]:
tracks_df.head(5)

,trackname,album,artist,length,popularity,danceability,acousticness,energy,instrumentalness,mode,liveness,loudness,speechiness,tempo,time_signature,valence
0,Unter meiner Haut,Unter meiner Haut,ELIF,195253,45,0.643,0.0582,0.711,0.000000,1,0.0509,-7.049,0.0640,92.014,4,0.363
1,Yes Indeed,Yes Indeed,Alicia Awa,180869,57,0.757,0.0685,0.529,0.001400,0,0.1200,-8.582,0.0502,128.030,4,0.177
2,HIGHWAY (feat. ELIF),HIGHWAY (feat. ELIF),Katja Krasavice,164062,70,0.618,0.5100,0.480,0.000059,0,0.1110,-10.135,0.0353,144.970,4,0.386
3,No Mercy,No Mercy,Genuva,155000,44,0.749,0.1840,0.597,0.000000,1,0.1770,-6.715,0.2330,88.164,4,0.822
4,Bye Bye,Bye Bye,Sarah Connor,194724,65,0.538,0.2090,0.696,0.000000,1,0.0923,-6.474,0.1370,163.867,4,0.534


In [4]:
dct = sp.playlist("43Fiv11oJLTBKtc2VcpmIy", fields=None, market=None, additional_types=('track', ))
result = sp.search(dct['tracks']['items'][0]['track']['artists'][0]['name'])
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

genres = artist["genres"]
artist = artist['name']
print(genres)
print (artist)

artist genres: ['dutch experimental electronic']
['dutch experimental electronic']
Amy Root


In [5]:
from tqdm import tqdm
import pandas as pd

In [6]:
rows = []
for t in tqdm(range(len(tracks_df))):
    track_row = []
    
    track = dct['tracks']['items'][t]['track']['name']
    artist = dct['tracks']['items'][t]['track']['artists'][0]['name']
    
    result = sp.search(dct['tracks']['items'][t]['track']['artists'][0]['name'])
    tr = result['tracks']['items'][0]
    art = sp.artist(tr["artists"][0]["external_urls"]["spotify"])
    
    genre = art["genres"]
    try:
        track_row = [artist, genre[0]]
    except:
        track_row = [artist]
    rows.append(track_row)
    
    
genre = pd.DataFrame.from_dict(rows)

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [00:15<00:00,  5.97it/s]


In [7]:
genre.columns = ['artist', 'genres']

In [8]:
genre

,artist,genres
0,ELIF,dutch experimental electronic
1,Alicia Awa,None
2,Katja Krasavice,frauenrap
3,Genuva,None
4,Sarah Connor,europop
...,...,...
88,MEL,colombian pop
89,Tayna,None
90,MEL,colombian pop
91,Emir Can İğrek,turkish pop


In [9]:
df = pd.merge(tracks_df,genre,  on = 'artist') #"artist" is the common identifier
df.head(5)

,trackname,album,artist,length,popularity,danceability,acousticness,energy,instrumentalness,mode,liveness,loudness,speechiness,tempo,time_signature,valence,genres
0,Unter meiner Haut,Unter meiner Haut,ELIF,195253,45,0.643,0.0582,0.711,0.0,1,0.0509,-7.049,0.064,92.014,4,0.363,dutch experimental electronic
1,Unter meiner Haut,Unter meiner Haut,ELIF,195253,45,0.643,0.0582,0.711,0.0,1,0.0509,-7.049,0.064,92.014,4,0.363,dutch experimental electronic
2,Unter meiner Haut,Unter meiner Haut,ELIF,195253,45,0.643,0.0582,0.711,0.0,1,0.0509,-7.049,0.064,92.014,4,0.363,dutch experimental electronic
3,Unter meiner Haut,Unter meiner Haut,ELIF,195253,45,0.643,0.0582,0.711,0.0,1,0.0509,-7.049,0.064,92.014,4,0.363,dutch experimental electronic
4,Unter meiner Haut,Unter meiner Haut,ELIF,195253,45,0.643,0.0582,0.711,0.0,1,0.0509,-7.049,0.064,92.014,4,0.363,dutch experimental electronic


In [10]:
grouped = df.groupby(['trackname','artist'], as_index=True).size()
grouped[grouped > 1].count()

38

In [11]:
df.drop_duplicates(subset=['trackname','artist'], inplace=True)

In [12]:
df.to_csv("playlist1.csv", sep = ',')